In [18]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timezone, timedelta

def extract_flight_data():
    """
    Extracts flight data from the OpenSky API for the last 24 hours 
    while handling API restrictions (max 2-hour query windows).
    
    Returns:
        pd.DataFrame: A DataFrame containing flight data.
    """

    # Load credentials from template.env
    load_dotenv(".env")

    USERNAME = os.getenv("OPENSKY_USERNAME")
    PASSWORD = os.getenv("OPENSKY_PASSWORD")

    if not USERNAME or not PASSWORD:
        raise ValueError("Missing OpenSky credentials. Check your template.env file.")

    # Define time range (last 24 hours) using timezone-aware datetime
    end_time = int(datetime.now(timezone.utc).timestamp())   # Current UTC time
    start_time = end_time - 86400  # 24 hours ago

    # Initialize empty list to store data
    all_flights = []

    # Loop through 2-hour intervals (to comply with OpenSky limits)
    interval = 7200  # 2 hours in seconds
    current_start = start_time

    while current_start < end_time:
        current_end = min(current_start + interval, end_time)  # Ensure we don't exceed the last timestamp
        
        # OpenSky API URL for 2-hour window
        url = f"https://opensky-network.org/api/flights/all?begin={current_start}&end={current_end}"

        # Fetch data from OpenSky API
        response = requests.get(url, auth=(USERNAME, PASSWORD))

        # Check for successful response
        if response.status_code == 200:
            flights = response.json()
            all_flights.extend(flights)  # Append new results to list
        else:
            print(f"Error {response.status_code}: {response.text}")
        
        # Move to next 2-hour window
        current_start = current_end

    # Convert collected data to DataFrame
    df_flights = pd.DataFrame(all_flights)

    return df_flights

# Run the function and display first 20 rows
df_flights = extract_flight_data()
df_flights.head(20)


,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
0,00810b,1741054449,FAOR,1741055152,FANL,ZSOEE,3984,180,5643,1817,65,61
1,0100a6,1741049325,None,1741050427,None,MSR676,0,0,0,0,0,0
2,0100df,1741052024,None,1741054725,None,MSR740,0,0,0,0,0,0
3,01010a,1741051078,None,1741051793,None,MSR634,0,0,0,0,0,0
4,010147,1741048277,None,1741049843,HE15,MSR812,0,0,9899,2598,0,7
5,0101be,1741055593,None,1741055664,None,MSR4413,0,0,0,0,0,0
6,0101cd,1741054476,None,1741055655,None,RBG570,0,0,0,0,0,0
7,01022c,1741033206,None,1741048721,EVKA,MSC901,0,0,26618,11799,0,0
8,01022e,1741052547,None,1741053665,None,MSC2450,0,0,0,0,0,0
9,01023d,1741041110,OKBK,1741049237,None,MSC408,1886,47,0,0,13,0


In [19]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timezone, timedelta
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, Boolean, MetaData, Table, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Function to extract flight data from OpenSky API
def extract_flight_data():


    # Load credentials from template.env
    load_dotenv(".env")
    USERNAME = os.getenv("OPENSKY_USERNAME")
    PASSWORD = os.getenv("OPENSKY_PASSWORD")
    if not USERNAME or not PASSWORD:
        raise ValueError("Missing OpenSky credentials. Check your template.env file.")

    # Define time range (last 24 hours) using timezone-aware datetime
    end_time = int(datetime.now(timezone.utc).timestamp())  # Current UTC time
    start_time = end_time - 86400  # 24 hours ago

    # Initialize empty list to store data
    all_flights = []

    # Loop through 2-hour intervals (to comply with OpenSky limits)
    interval = 7200  # 2 hours in seconds
    current_start = start_time
    while current_start < end_time:
        current_end = min(current_start + interval, end_time)  # Ensure we don't exceed the last timestamp
        
        # OpenSky API URL for 2-hour window
        url = f"https://opensky-network.org/api/flights/all?begin={current_start}&end={current_end}"
        
        # Fetch data from OpenSky API
        response = requests.get(url, auth=(USERNAME, PASSWORD))
        
        # Check for successful response
        if response.status_code == 200:
            flights = response.json()
            all_flights.extend(flights)  # Append new results to list
        else:
            print(f"Error {response.status_code}: {response.text}")
        
        # Move to next 2-hour window
        current_start = current_end

    # Convert collected data to DataFrame
    df_flights = pd.DataFrame(all_flights)
    return df_flights

# Database connection and setup
def setup_database():
    """
    Set up database connection and create the flight_data table if it doesn't exist.
    Returns:
        tuple: (engine, session) for database operations
    """
    # Load database credentials
    load_dotenv(".env")
    DB_USERNAME = os.getenv("DB_USER")  # Changed to match your .env
    DB_PASSWORD = os.getenv("DB_PASSWORD")
    DB_HOST = os.getenv("DB_HOST", "db-raw-opensky.cjeesaow22lr.eu-central-1.rds.amazonaws.com")
    DB_PORT = os.getenv("DB_PORT", "5432")
    DB_NAME = os.getenv("DB_NAME", "db-raw-opensky")
    
    try:
        # Try to create PostgreSQL engine
        print("Attempting to connect to PostgreSQL...")
        connection_string = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
        engine = create_engine(connection_string)
        print(f"Successfully connected to {DB_NAME} at {DB_HOST}")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        
        # Fall back to SQLite as a last resort
        print("Falling back to SQLite database...")
        sqlite_path = "flight_data.db"
        engine = create_engine(f"sqlite:///{sqlite_path}")
        print(f"Using SQLite database at {sqlite_path}")
    
    # Create base class for declarative models
    Base = declarative_base()
    
    # Define the flight_data table schema
    class FlightData(Base):
        __tablename__ = 'flight_data'
        
        id = Column(Integer, primary_key=True, autoincrement=True)
        icao24 = Column(String(24))
        firstSeen = Column(Integer)
        estDepartureAirport = Column(String(4))
        lastSeen = Column(Integer)
        estArrivalAirport = Column(String(4))
        callsign = Column(String(8))
        estDepartureAirportHorizDistance = Column(Integer)
        estDepartureAirportVertDistance = Column(Integer)
        estArrivalAirportHorizDistance = Column(Integer)
        estArrivalAirportVertDistance = Column(Integer)
        departureAirportCandidatesCount = Column(Integer)
        arrivalAirportCandidatesCount = Column(Integer)
        
        def __repr__(self):
            return f"<Flight(icao24='{self.icao24}', callsign='{self.callsign}')>"
    
    # Create all tables if they don't exist
    Base.metadata.create_all(engine)
    
    # Create a session factory
    Session = sessionmaker(bind=engine)
    session = Session()
    
    return engine, session, FlightData

def main():
    """
    Main function to extract flight data and load it into the database.
    """
    print("Starting flight data ETL process...")
    
    # Extract flight data from OpenSky API
    print("Extracting flight data from OpenSky API...")
    df_flights = extract_flight_data()
    print(f"Retrieved {len(df_flights)} flight records.")
    
    # Setup database connection and get the FlightData model
    print("Setting up database connection...")
    engine, session, FlightData = setup_database()
    
    # Check if table already exists (it should have been created in setup_database)
    print("Checking if flight_data table exists...")
    metadata = MetaData()
    metadata.reflect(bind=engine)
    
    # Transform and load data into the database
    print("Loading data into the flight_data table...")
    
    # Handle potential missing columns in the DataFrame
    required_columns = [
        'icao24', 'firstSeen', 'estDepartureAirport', 'lastSeen', 
        'estArrivalAirport', 'callsign', 'estDepartureAirportHorizDistance',
        'estDepartureAirportVertDistance', 'estArrivalAirportHorizDistance', 
        'estArrivalAirportVertDistance', 'departureAirportCandidatesCount',
        'arrivalAirportCandidatesCount'
    ]
    
    # Create empty columns if they don't exist
    for col in required_columns:
        if col not in df_flights.columns:
            df_flights[col] = None
    
    # Bulk insert approach for better performance
    flight_records = []
    for _, row in df_flights.iterrows():
        # Convert row to dictionary, handling any missing columns
        flight_data = {}
        for col in required_columns:
            flight_data[col] = row.get(col) if col in row else None
        
        # Create FlightData object
        flight_records.append(FlightData(**flight_data))
    
    try:
        # Add all records to the session
        session.bulk_save_objects(flight_records)
        # Commit the transaction
        session.commit()
        print(f"Successfully loaded {len(flight_records)} records into the flight_data table.")
    except Exception as e:
        session.rollback()
        print(f"Error loading data into database: {e}")
        print(f"Exception details: {str(e)}")
    finally:
        session.close()
    
    print("ETL process completed.")

if __name__ == "__main__":
    main()

Starting flight data ETL process...
Extracting flight data from OpenSky API...
Retrieved 16191 flight records.
Setting up database connection...
Attempting to connect to PostgreSQL...
Failed to connect to PostgreSQL: No module named 'psycopg2'
Falling back to SQLite database...
Using SQLite database at flight_data.db
Checking if flight_data table exists...
Loading data into the flight_data table...


/var/folders/k5/0ckgxv2x46n7_p0qwn2h0hhh0000gn/T/ipykernel_42518/4097074204.py:85: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Successfully loaded 16191 records into the flight_data table.
ETL process completed.


In [17]:
# Import required libraries
import os
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import sys

# Set the number of recent records to display
limit = 10  # Change this value to view more or fewer records

# Try to install and import psycopg2 if not present
try:
    import psycopg2
    print("psycopg2 already installed")
except ImportError:
    print("psycopg2 not found, trying to install...")
    try:
        # Use system pip to install
        !pip install psycopg2-binary
        import psycopg2
        print("psycopg2-binary installed successfully")
    except Exception as e:
        print(f"Failed to install psycopg2-binary: {e}")
        print("Will try to use SQLite as fallback if needed")

# Import SQLAlchemy components
try:
    from sqlalchemy import create_engine, MetaData, Table, select, desc, func
except ImportError as e:
    print(f"Error importing SQLAlchemy components: {e}")
    !pip install sqlalchemy
    from sqlalchemy import create_engine, MetaData, Table, select, desc, func

def connect_to_database():
    """Connect to the flight_data database."""
    # Load environment variables
    load_dotenv(".env")
    
    # Get database credentials
    DB_USERNAME = os.getenv("DB_USER")
    DB_PASSWORD = os.getenv("DB_PASSWORD")
    DB_HOST = os.getenv("DB_HOST", "db-raw-opensky.cjeesaow22lr.eu-central-1.rds.amazonaws.com")
    DB_PORT = os.getenv("DB_PORT", "5432")
    DB_NAME = os.getenv("DB_NAME", "db-raw-opensky")
    
    try:
        # Check if psycopg2 is available
        import psycopg2
        
        # Create the connection string and engine for PostgreSQL
        print("Attempting to connect to PostgreSQL database...")
        connection_string = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
        engine = create_engine(connection_string)
        
        # Test connection
        with engine.connect() as conn:
            print(f"Successfully connected to {DB_NAME} at {DB_HOST}")
            
        return engine
    
    except ImportError:
        print("psycopg2 not available, trying alternative approaches...")
        
        # Try pg8000 as an alternative PostgreSQL driver
        try:
            # Install pg8000 if not present
            !pip install pg8000
            
            connection_string = f"postgresql+pg8000://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
            engine = create_engine(connection_string)
            
            # Test connection
            with engine.connect() as conn:
                print(f"Successfully connected using pg8000 to {DB_NAME} at {DB_HOST}")
                
            return engine
        
        except Exception as e:
            print(f"Failed to connect with pg8000: {e}")
            
            # Fall back to SQLite
            print("Falling back to SQLite database...")
            sqlite_path = "flight_data.db"
            engine = create_engine(f"sqlite:///{sqlite_path}")
            print(f"Using SQLite database at {sqlite_path}")
            
            # Check if flight_data table exists in SQLite
            metadata = MetaData()
            metadata.reflect(bind=engine)
            if 'flight_data' not in metadata.tables:
                print("No flight_data table found in SQLite. This is expected if you haven't used SQLite before.")
                print("You'll need to connect to your PostgreSQL database to see the flight data.")
            
            return engine
    
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

def get_recent_flights(engine, limit=10):
    """Retrieve the most recent flight records from the database."""
    if not engine:
        print("No database engine available")
        return None, 0
    
    try:
        # Create metadata object
        metadata = MetaData()
        metadata.reflect(bind=engine)
        
        # Check if flight_data table exists
        if 'flight_data' not in metadata.tables:
            print("The flight_data table does not exist in the database.")
            return None, 0
        
        # Access the flight_data table
        flight_table = metadata.tables['flight_data']
        
        # Create query to get most recent records (ordered by id descending)
        query = select(flight_table).order_by(desc(flight_table.c.id)).limit(limit)
        
        # Execute query and fetch results
        with engine.connect() as conn:
            result = conn.execute(query)
            records = result.fetchall()
            
            # Get column names
            columns = result.keys()
            
            # Convert to DataFrame for better display
            df = pd.DataFrame(records, columns=columns)
            
            # Count total records in the table
            count_query = select(func.count()).select_from(flight_table)
            total_count = conn.execute(count_query).scalar()
            
            return df, total_count
    
    except Exception as e:
        print(f"Error retrieving flight data: {e}")
        print(f"Exception type: {type(e).__name__}")
        print(f"Exception details: {str(e)}")
        return None, 0

def format_timestamp(timestamp):
    """Convert Unix timestamp to readable date format."""
    if pd.isna(timestamp) or timestamp is None:
        return "N/A"
    try:
        return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        return str(timestamp)

# Connect to the database
engine = connect_to_database()

if engine:
    # Get the data
    df, total_count = get_recent_flights(engine, limit)
    
    if df is None or df.empty:
        print("No flight data found.")
    else:
        print(f"\nTotal records in flight_data table: {total_count}")
        print(f"Displaying the {len(df)} most recent records:\n")
        
        # Convert Unix timestamps to readable format for display
        if 'firstSeen' in df.columns:
            df['firstSeen_formatted'] = df['firstSeen'].apply(format_timestamp)
        if 'lastSeen' in df.columns:
            df['lastSeen_formatted'] = df['lastSeen'].apply(format_timestamp)
        
        # Select columns to display
        display_columns = ['id', 'icao24', 'callsign', 'firstSeen_formatted', 'lastSeen_formatted', 
                          'estDepartureAirport', 'estArrivalAirport']
        
        # Ensure all selected columns exist
        display_columns = [col for col in display_columns if col in df.columns]
        
        # Display the data
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        display(df[display_columns])
        
        # Show some stats
        print("\nExample Flight Stats:")
        
        # Most common departure airports
        if 'estDepartureAirport' in df.columns:
            dep_airports = df['estDepartureAirport'].value_counts().head(3)
            if not dep_airports.empty:
                print("\nMost common departure airports in this dataset:")
                for airport, count in dep_airports.items():
                    if pd.notna(airport) and airport:
                        print(f"  {airport}: {count} flights")
        
        # Most common arrival airports
        if 'estArrivalAirport' in df.columns:
            arr_airports = df['estArrivalAirport'].value_counts().head(3)
            if not arr_airports.empty:
                print("\nMost common arrival airports in this dataset:")
                for airport, count in arr_airports.items():
                    if pd.notna(airport) and airport:
                        print(f"  {airport}: {count} flights")

psycopg2 not found, trying to install...

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Failed to install psycopg2-binary: No module named 'psycopg2'
Will try to use SQLite as fallback if needed
psycopg2 not available, trying alternative approaches...

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Failed to connect with pg8000: No module named 'pg8000'
Falling back to SQLite database...
Using SQLite database at flight_data.db

Total records in flight_data table: 16331
Displaying the 10 most recent records:



,id,icao24,callsign,firstSeen_formatted,lastSeen_formatted,estDepartureAirport,estArrivalAirport
0,16331,e8027c,LPE2299,2025-03-03 19:54:33,2025-03-03 20:20:58,None,SPUC
1,16330,e8020c,LPE2294,2025-03-03 19:44:22,2025-03-03 20:11:20,None,SPBA
2,16329,e49908,TAM3836,2025-03-03 20:06:59,2025-03-03 20:41:07,SBBP,SWYV
3,16328,e495cd,GOL2094,2025-03-03 17:27:49,2025-03-03 20:12:22,SBGL,SBFZ
4,16327,c823ec,ZKROK,2025-03-03 20:56:28,2025-03-03 21:31:10,NZQN,NZQN
5,16326,c821f8,ANZ035M,2025-03-03 20:06:25,2025-03-03 20:33:06,NZAA,NZRU
6,16325,c81bb3,ANZ830L,2025-03-03 20:51:55,2025-03-03 21:11:39,NZCH,NZMC
7,16324,c082f1,SDE8O0,2025-03-03 19:19:02,2025-03-03 20:48:51,KBOI,KPSP
8,16323,c080cd,WEN3242,2025-03-03 18:59:27,2025-03-03 20:13:17,CYYC,CKU6
9,16322,c0807f,WEN3121,2025-03-03 19:53:25,2025-03-03 20:24:47,CYEG,CFF2



Example Flight Stats:

Most common departure airports in this dataset:
  SBBP: 1 flights
  SBGL: 1 flights
  NZQN: 1 flights

Most common arrival airports in this dataset:
  SPUC: 1 flights
  SPBA: 1 flights
  SWYV: 1 flights
